In [1]:
import numpy as np
import numpy as npX
import pandas as pd
import seaborn as sns
import re
import json

In [2]:
vec = pd.read_csv("df_char.csv")


#df1 = df.set_index('timestamp')




vec



,tweet_id,priority,category,postCategories_x,eventID,eventType,matchedName
0,211565974422425600,0.75,serviceavailable,19,fireColorado2012,Unknown,[]
1,211654415503990784,0.50,news,15,fireColorado2012,Unknown,[]
2,211681309368655872,0.25,news,15,fireColorado2012,Unknown,['informationwanted']
3,211685621125742592,0.25,official,16,fireColorado2012,Unknown,['volunteer']
4,211877049147736064,0.25,factoid,7,fireColorado2012,Unknown,['donations']
...,...,...,...,...,...,...,...
938,396336012726525952,0.25,news,15,laAirportShooting2013,Unknown,NaN
939,396336079856345088,0.25,news,15,laAirportShooting2013,Unknown,NaN
940,396336243442589696,0.25,news,15,laAirportShooting2013,Unknown,NaN
941,396336297968562176,0.25,factoid,7,laAirportShooting2013,Unknown,NaN


In [3]:
res = pd.read_csv("results_merged.csv")

res['tweet_id'] = pd.to_numeric(res['tweet_id'].apply(lambda x: re.sub(',', '.', str(x))))

#res.reset_index()

res.head()



,tweet_id,priority,postCategories_x,result
0,2.115660e+17,0.75,19.0,14
1,2.116544e+17,0.50,15.0,14
2,2.116813e+17,0.25,15.0,14
3,2.116856e+17,0.25,16.0,14
4,2.118770e+17,0.25,7.0,14


In [4]:
df_merged = pd.merge(res, vec, left_on = 'tweet_id', right_on = 'tweet_id', how = 'inner')

df_merged

,tweet_id,priority_x,postCategories_x_x,result,priority_y,category,postCategories_x_y,eventID,eventType,matchedName
0,2.115660e+17,0.75,19.0,14,0.75,serviceavailable,19,fireColorado2012,Unknown,[]
1,2.116544e+17,0.50,15.0,14,0.50,news,15,fireColorado2012,Unknown,[]
2,2.116813e+17,0.25,15.0,14,0.25,news,15,fireColorado2012,Unknown,['informationwanted']
3,2.116856e+17,0.25,16.0,14,0.25,official,16,fireColorado2012,Unknown,['volunteer']
4,2.118770e+17,0.25,7.0,14,0.25,factoid,7,fireColorado2012,Unknown,['donations']
...,...,...,...,...,...,...,...,...,...,...
875,3.963359e+17,0.25,15.0,5,0.25,news,15,laAirportShooting2013,Unknown,NaN
876,3.963361e+17,0.25,15.0,5,0.25,news,15,laAirportShooting2013,Unknown,NaN
877,3.963362e+17,0.25,15.0,5,0.25,news,15,laAirportShooting2013,Unknown,NaN
878,3.963363e+17,0.25,7.0,5,0.25,factoid,7,laAirportShooting2013,Unknown,NaN


In [5]:
with open('../../../data/raw/data/2020/2020-A/types/TRECIS-2020-ITypes-Task1.json', encoding='utf-8') as ontologyJSONFile:    
    ontologyJSON = json.load(ontologyJSONFile)

informationTypes2Index = {} # category -> numerical index
informationTypesShort2Index = {} # category short form (e.g. Report-EmergingThreats vs. EmergingThreats) -> numerical index
informationTypeIndex = 0
for informationTypeJSON in ontologyJSON["informationTypes"]:
    informationTypeId = informationTypeJSON["id"]
    informationTypes2Index[informationTypeId] = informationTypeIndex
    informationTypesShort2Index[informationTypeId.split("-")[1]] = informationTypeIndex
    informationTypeIndex = informationTypeIndex + 1
informationTypes2Index

{'Request-GoodsServices': 0,
 'Request-SearchAndRescue': 1,
 'Request-InformationWanted': 2,
 'CallToAction-Volunteer': 3,
 'CallToAction-Donations': 4,
 'CallToAction-MovePeople': 5,
 'Report-FirstPartyObservation': 6,
 'Report-ThirdPartyObservation': 7,
 'Report-Weather': 8,
 'Report-Location': 9,
 'Report-EmergingThreats': 10,
 'Report-NewSubEvent': 11,
 'Report-MultimediaShare': 12,
 'Report-ServiceAvailable': 13,
 'Report-Factoid': 14,
 'Report-Official': 15,
 'Report-News': 16,
 'Report-CleanUp': 17,
 'Report-Hashtags': 18,
 'Report-OriginalEvent': 19,
 'Other-ContextualInformation': 20,
 'Other-Advice': 21,
 'Other-Sentiment': 22,
 'Other-Discussion': 23,
 'Other-Irrelevant': 24}

In [6]:
eventidTopicidMap =	{
  "athensEarthquake2020": "35",
  "baltimoreFlashFlood2020": "36",
  "brooklynBlockPartyShooting2020": "37",
  "daytonOhioShooting2020": "38",
  "elPasoWalmartShooting2020": "39",
  "gilroygarlicShooting2020": "40",
  "hurricaneBarry2020": "41",
  "indonesiaEarthquake2020": "42",
  "keralaFloods2020": "43",
  "myanmarFloods2020": "44",
  "papuaNewguineaEarthquake2020": "45",
  "siberianWildfires2020": "46",
  "typhoonKrosa2020": "47",
  "typhoonLekima2020": "48",
  "whaleyBridgeCollapse2020": "49"
}
eventidTopicidMap

{'athensEarthquake2020': '35',
 'baltimoreFlashFlood2020': '36',
 'brooklynBlockPartyShooting2020': '37',
 'daytonOhioShooting2020': '38',
 'elPasoWalmartShooting2020': '39',
 'gilroygarlicShooting2020': '40',
 'hurricaneBarry2020': '41',
 'indonesiaEarthquake2020': '42',
 'keralaFloods2020': '43',
 'myanmarFloods2020': '44',
 'papuaNewguineaEarthquake2020': '45',
 'siberianWildfires2020': '46',
 'typhoonKrosa2020': '47',
 'typhoonLekima2020': '48',
 'whaleyBridgeCollapse2020': '49'}

```TRECIS-CTIT-H-Test-046	Q0	2.115659744224256e+17	0	0.75	19.0	marksrun```

In [8]:
priority_scorer = {
    'Critical' : 0.75,
    'High' : 0.75,
    'Medium' : 0.50,
    'Low' : 0.25,
    'Unknown' : 0.0,
}


categoriser = {
    0 : 'Advice',
    1 : 'CleanUp',
    2 : 'ContextualInformation',
    3 : 'Discussion',
    4 : 'Donations',
    5 : 'EmergingThreats',
    6 : 'Factoid',
    7 : 'FirstPartyObservation',
    8 : 'Hashtags',
    9 : 'InformationWanted',
    10 : 'Irrelevant',
    11 : 'MovePeople',
    12 : 'MultimediaShare',
    13 : 'NewSubEventst',
    14 : 'News',
    15 : 'Official',
    16 : 'OriginalEvent',
    17 : 'Sentiment',
    18 : 'ServiceAvailable',
    19 : 'ThirdPartyObservation',
    20 : 'Unknown',
    21 : 'Volunteer', 
    22 : 'Weather', 
    23 : 'test', 
    24 : 'missing', 

}

# write to .run file
with open("marks2.run" , "w") as out_file:
    for row in df_merged.drop_duplicates(subset="tweet_id").itertuples():
       # print("row:", row)
        content = [
            row.eventID,
            "Q0",           # static `Q0`
            np.int64(row.tweet_id),   # tweet_id
            getattr(row, 'Index'),
            row.priority_x, 
            "['" + categoriser[row.result] + "']",
            "marksrun2"
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")